In [1]:
import numpy as np
import pandas as pd

In [3]:
import numpy as np
import itertools

# -------------------------------
# PARAMETERS AND STATE STRUCTURE
# -------------------------------
# Parameters dictionary now includes:
#   n             : number of players
#   T             : number of periods
#   delta         : delay cost parameter (first component)
#   delta_prime   : delay cost parameter (second component)
#   gamma         : cost per unit for borrowing using traditional collateral
#   phi           : cost per unit for borrowing using incoming claims
#   chi           : cost per unit for borrowing unsecured credit
#   has_collateral: list or array of booleans (length n) indicating if a player has collateral available

def state_to_key(t, balances, obligations):
    # Convert state (period, balances, obligations) to a tuple key for caching.
    return (t, tuple(balances.round(decimals=6)), tuple(obligations.round(decimals=6).flatten()))

# -------------------------------
# UPDATE STATE FUNCTION
# -------------------------------
def update_state(obligations, balances, actions, params, is_last_period=False):
    """
    Update the state given:
      - obligations: current obligations matrix (n x n)
      - balances: current balances vector (length n)
      - actions: list of actions for each player, where each action is 'pay' or 'delay'
      - params: dictionary of parameters
      - is_last_period: Boolean flag to indicate if automatic settlement applies (last period)
    
    Returns:
      - immediate_total_cost: sum of costs incurred this period across players
      - new_obligations: updated obligations matrix (for next period)
      - new_balances: updated balances vector (for next period)
    """
    n = params['n']
    delta = params['delta']
    delta_prime = params['delta_prime']
    gamma = params['gamma']
    phi = params['phi']
    chi = params['chi']
    has_collateral = params['has_collateral']  # expected to be an array/list of booleans of length n
    
    new_balances = np.zeros(n)
    new_obligations = np.zeros((n, n))
    cost_individual = np.zeros(n)
    
    # Calculate inflows: for each player i, sum obligations from players who decide to pay.
    inflows = np.zeros(n)
    for j in range(n):
        if actions[j] == 'pay':
            inflows += obligations[j, :]  # obligations[j,j] remains zero.
    
    for i in range(n):
        if actions[i] == 'pay':
            # Player pays all obligations.
            outflow = np.sum(obligations[i, :])
            net = balances[i] + inflows[i] - outflow
            # Determine shortfall and choose borrowing option:
            shortfall = max(0.0, -net)
            if shortfall > 0:
                if has_collateral[i]:
                    if phi < gamma and phi < chi:
                        borrow_cost = shortfall * phi
                    elif gamma < chi:
                        borrow_cost = shortfall * gamma
                    else:
                        borrow_cost = shortfall * chi
                else:
                    borrow_cost = shortfall * chi
            else:
                borrow_cost = 0.0
            
            cost_individual[i] = borrow_cost
            new_balances[i] = max(net, 0)
            # Obligations are cleared when paying.
        elif actions[i] == 'delay':
            # Player delays; obligations are not cleared.
            current_obligation = np.sum(obligations[i, :])
            delay_cost = (delta + delta_prime) * current_obligation
            net = balances[i] + inflows[i] - delay_cost
            shortfall = max(0.0, -net)
            if shortfall > 0:
                if has_collateral[i]:
                    if phi < gamma and phi < chi:
                        borrow_cost = shortfall * phi
                    elif gamma < chi:
                        borrow_cost = shortfall * gamma
                    else:
                        borrow_cost = shortfall * chi
                else:
                    borrow_cost = shortfall * chi
            else:
                borrow_cost = 0.0
            
            cost_individual[i] = delay_cost + borrow_cost
            new_balances[i] = max(net, 0)
            if is_last_period:
                new_obligations[i, :] = 0  # obligations vanish at final settlement.
            else:
                new_obligations[i, :] = obligations[i, :]
        else:
            raise ValueError("Action must be either 'pay' or 'delay'.")
    
    immediate_total_cost = np.sum(cost_individual)
    return immediate_total_cost, new_obligations, new_balances

# -------------------------------
# GAME SEARCH TREE: Recursive Backward Induction
# -------------------------------
def search(t, obligations, balances, arrivals_list, params, memo=None):
    """
    Recursive search for the first best solution (minimizing total cost).
    Returns:
      - best_total_cost: minimal cumulative cost from period t to end.
      - best_actions_path: list of action profiles (one per period) leading to that cost.
    """
    if memo is None:
        memo = {}
    
    key = state_to_key(t, balances, obligations)
    if key in memo:
        return memo[key]
    
    n = params['n']
    T = params['T']
    
    # Base case: if t equals T, we're past the last period.
    if t == T:
        return 0, []
    
    # Add new arrivals to current obligations.
    arrivals = arrivals_list[t]
    current_obligations = obligations + arrivals
    
    best_total_cost = np.inf
    best_actions_path = None
    
    for actions in itertools.product(['pay', 'delay'], repeat=n):
        is_last_period = (t == T - 1)
        immediate_cost, next_obligations, next_balances = update_state(current_obligations, balances, actions, params, is_last_period=is_last_period)
        future_cost, future_path = search(t + 1, next_obligations, next_balances, arrivals_list, params, memo)
        total_cost = immediate_cost + future_cost
        
        if total_cost < best_total_cost:
            best_total_cost = total_cost
            best_actions_path = [actions] + future_path
            
    memo[key] = (best_total_cost, best_actions_path)
    return best_total_cost, best_actions_path


def first_best_solution(arrivals_list, params):
    n = params['n']
    obligations = np.zeros((n, n))
    balances = np.zeros(n)
    
    total_cost, actions_path = search(0, obligations, balances, arrivals_list, params)
    return total_cost, actions_path


In [6]:

def generate_arrivals_list(n, T, p_t_list, seed=None):
    """
    Generate a list of arrival matrices for n players over T periods.
    
    Parameters:
        n (int): Number of players
        T (int): Number of periods
        p_t_list (list of float): Probability of a transaction per period (length T)
        seed (int): Optional random seed for reproducibility

    Returns:
        arrivals_list (list of np.ndarray): List of T n x n arrival matrices
    """
    if seed is not None:
        np.random.seed(seed)
    
    arrivals_list = []
    
    for t in range(T):
        p_t = p_t_list[t]
        A = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    A[i, j] = 1 if np.random.rand() < p_t else 0
        arrivals_list.append(A)
    
    return arrivals_list

In [ ]:
n = 3
T = 10
params = {
    'n': n,           # number of players
    'T': T,           # number of periods
    'delta': 0.5,     # delay cost parameter
    'delta_prime': 0.2,  # additional delay cost parameter
    'gamma': 1.0,     # borrowing cost for traditional collateral
    'phi': 0.8,       # borrowing cost using incoming claims
    'chi': 1.2,       # borrowing cost for unsecured credit
    'has_collateral': [True] * n
}

n = params['n']
T = params['T']
p_t_list = [0.8]*T 

arrivals_list = generate_arrivals_list(n, T, p_t_list, seed=42)

total_cost, actions_path = first_best_solution(arrivals_list, params)

print("Best total cost (lowest total cost across players):", total_cost)
print("Actions per period:")
for t, actions in enumerate(actions_path):
    print(f"Period {t}: {actions}")


Best total cost (lowest total cost across players): 2.4000000000000004
Actions per period (each tuple is the actions for players 0 and 1):
Period 0: ('pay', 'pay', 'pay')
Period 1: ('pay', 'pay', 'pay')
Period 2: ('pay', 'pay', 'pay')
Period 3: ('pay', 'pay', 'pay')
Period 4: ('pay', 'pay', 'pay')
Period 5: ('pay', 'pay', 'pay')
Period 6: ('pay', 'pay', 'pay')
Period 7: ('pay', 'pay', 'pay')
Period 8: ('pay', 'pay', 'pay')
Period 9: ('pay', 'pay', 'pay')
